<a href="https://colab.research.google.com/github/Chocoding1/Machine_Learning_Deep_Learning/blob/main/05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **교차 검증과 그리드 서치**

지금까지는 모델의 성능을 확인하기 위해 테스트 세트를 사용했다. 그러나 성능 확인을 위해 테스트 세트를 자꾸 사용하다 보면 모델이 점점 테스트 세트에 맞춰진다.<br>
테스트 세트로 모델의 일반화 성능을 올바르게 예측하려면 가능한 한 테스트 세트를 사용하지 말아야 한다. 모델을 만들고 나서 마지막에 딱 한 번만 사용하는 것이 좋다.

## 검증 세트

테스트 세트를 사용하지 않으면 모델이 과대 적합인지 과소 적합인지 판단하기 어렵다.<br>
테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 나누는 것이다.<br>
이 데이터를 **검증 세트**라고 한다.<br>

1절에서 전체 데이터 중 20%를 테스트 세트로 만들고 나머지를 훈련 세트로 만들었는데, 검증 세트는 이 훈련 세트에서 다시 20%를 떼어 내어 만든다.<br>
1. 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가
2. 이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 선정
3. 결정된 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련
4. 마지막으로 테스트 세트에서 최종 점수 평가

In [2]:
# 데이터 세트 불러오기
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
# 입력 데이터와 타깃 데이터 설정하고 훈련을 위해 넘파이 배열로 변경
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
# 훈련 세트와 데이터 세트 분리
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [5]:
# 훈련 세트를 다시 분리하여 검증 세트 만들기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [6]:
# 훈련 세트, 검증 세트, 테스트 세트 크기 확인
print(sub_input.shape, val_input.shape, test_input.shape)

(4157, 3) (1040, 3) (1300, 3)


In [7]:
# 훈련 세트와 검증 세트를 사용해 모델 훈련
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


위 모델은 확실히 과대 적합 -> 매개변수를 바꾸며 더 좋은 모델을 찾아야 한다.

## 교차 검증

검증 세트를 만드느라 훈련 세트가 줄었는데, 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어지는 만큼 검증 세트로 인해 성능이 좋지 못한 모델이 만들어질 수도 있다. 그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 불안정할 것이다.<br>
이 때 **교차 검증**을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.<br>
교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다. 그 평가한 여러 점수들을 평균하여 최종 검증 점수를 얻는다.

note) **k-폴드 교차 검증**<br>
훈련 세트를 k개로 나누어서 교차 검증을 수행하는 것으로 검증 세트를 계속해서 변경하며 최종적으로 k개의 서로 다른 검증 세트를 가지고 검증할 수 있다.<br>
이렇게 하면 데이터의 80~90%까지 훈련에 사용할 수 있다.<br>
검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각할 수 있다.<br>
보통 5-폴드 교차 검증이나 10-폴드 교차 검증을 많이 사용한다.

사이킷런에는 cross_validate()라는 교차 검증 함수가 존재한다.<br>
사용법) 평가할 모델 객체를 첫 번째 매개변수로 전달 -> 앞처럼 직접 검증 세트를 떼어내지 않고 훈련 세트 전체를 cross_validate() 함수에 전달<br>
cross_validate() 함수는 기본적으로 5-폴드 교차 검증 수행(cv 매개변수에서 폴드 수 변경 가능)<br>

In [8]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00809836, 0.00713062, 0.00752068, 0.00747609, 0.007092  ]), 'score_time': array([0.00111628, 0.00099611, 0.00105119, 0.00106955, 0.00107265]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


반환값) fit_time, score_time, test_score 키를 가진 딕셔너리 반환<br>
처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미<br>
마지막 test_score키에 교차 검증의 점수가 나오며, 교차 검증의 최종 점수는 test_score 키에 담긴 점수들을 평균하여 얻는다.

In [9]:
# 최종 교차 검증 점수 출력
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


주의할 점) cross_validate()는 훈련 세트를 섞어서 폴드를 나누지 않는다.<br>
앞의 train_test_split() 함수는 알아서 전체 데이터를 섞은 후에 나눠줬지만, cross_validate() 함수는 그렇지 않기 때문에 훈련 세트를 섞기 위해 분할기(splitter)를 지정해야 한다.

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해준다.<br>
cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용, 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다.

In [10]:
# 위에서 수행한 교차 검증은 다음 코드와 동일
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


위의 결과와 동일

In [11]:
# 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면?
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

하이퍼파라미터 : 머신러닝 모델이 학습하는 파라미터인 모델 파라미터와 달리, 모델이 학습할 수 없어 사용자가 지정해야만 하는 파라미터<br>

<하이퍼파라미터 튜닝 작업의 진행><br>
먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련한다. 그 다음 검증 세트의 점수나 교차 검증을 통해 매개변수를 조금씩 바꿔 본다.<br>

note) AutoML : 사람의 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술

### 그리드 서치

결정 트리 모델에서 최적의 max_depth 값을 찾았다고 가정하자. 그 다음 max_depth를 최적의 값으로 고정하고 min_samples_split을 바꿔가며 최적의 값을 찾는 방식으로 한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾아도 될까?<br>
안 된다. max_depth의 최적값은 min_samples_split 매개변수의 값이 바뀌면 함께 달라진다. 즉 이 두 매개변수를 동시에 바꿔가며 최적의 값을 찾아야 한다.<br>
이런 과정을 for문으로 직접 구현할 수도 있지만 사이킷런에서 제공하는 **그리드 서치**를 사용하면 편리하다.<br>

사이킷런의 GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다.(별도로 cross_validate() 함수 호출할 필요 x)

In [12]:
# 기본 매개변수를 사용한 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값 찾기
# GridSearchCV 클래스를 임포트하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 생성
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} # 0.0001부터 0.0005까지 0.0001씩 증가하는 5개의 값을 시도

In [13]:
# GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체 생성
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # n_jobs : 병렬 실행에 사용할 CPU 코어 수를 지정(기본값 : 1, -1로 지정하면 시스템에 있는 모든 코어 사용)

결정 트리 클래스의 객체를 생성하자마자 바로 전달한 후, gs 객체에 fit() 메서드를 호출한다. 이 메서드를 호출하면 그리드 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행한다.(값이 5개니까)<br>

GridSearchCV의 cv 매개변수 기본값은 5이므로 min_impurity_decrease 값마다 5-폴드 교차 검증을 수행한다. 결국 5 x 5 = 25개의 모델을 훈련하는 것

In [14]:
# 모델 훈련
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 했던 것과 달리,<br>
그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다.<br>
이 모델은 gs 객체의 best_estimator_ 속성에 저장되어 있으며, 해당 모델을 일반 결정 트리처럼 똑같이 사용할 수 있다.

In [17]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장되어 있다.

In [18]:
# 최적의 매개변수 출력
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score' 키에 저장되어 있다.

In [19]:
# 5번의 교차 검증으로 얻은 점수 출력
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


예상한 대로 첫 번째 점수가 가장 크다.<br>
하지만 일일이 눈으로 확인하는 것보다 넘파이 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다.<br>
그 다음 이 인덱스를 사용해 params 키에 저장된 매개변수를 출력할 수 있다. 이 값이 최상의 검증 점수를 만든 매개변수 조합이다.

In [22]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(best_index, gs.cv_results_['params'][best_index])

0 {'min_impurity_decrease': 0.0001}


<그리드 서치 과정 정리>
1. 탐색할 매개변수 지정
2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합 찾기(해당 조합은 그리드 서치 객체에 저장)
3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아닌) 전체 훈련 세트를 사용해 최종 모델을 훈련(해당 모델도 그리드 서치 객체에 저장)

더 복잡한 매개변수 조합 탐색<br>
1. min_impurity_decrease : 노드를 분할하기 위한 불순도 감소 최소량 지정
2. max_depth : 트리의 깊이 제한
3. min_samples_split : 노드를 나누기 위한 최소 샘플 수 지정

In [23]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001), 'max_depth' : range(5, 20, 1), 'min_samples_split' : range(2, 100, 10)}

np.arange() : 첫 번째 매개변수 값에서 시작하여 두 번째 매개변수 값에 도달할 때까지 세 번째 매개변수를 계속 더한 배열 생성<br>
파이썬의 range() 함수도 동일<br>

min_impurity_decrease의 원소 수는 9개, max_depth 원소 수는 15개, min_samples_split의 원소 수는 10개<br>
따라서 이 매개변수들로 수행할 교차 검증 횟수는 9 x 15 x 10 = 1,350개이다. 또한 기본 5-폴드 교차 검증을 수행하므로 만들어지는 모델의 수는 6,750개이다.

In [24]:
# 그리드 서치 실행
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [25]:
# 최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [26]:
# 최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

매개변수의 값이 수치일 떄 값의 범위나 간격을 미리 정하기 어려울 수도 있다. 또한 너무 많은 매개변수 조건이 있어서 그리드 서치 수행 시간이 오래 걸릴 수 있는데, 이럴 때 **랜덤 서치**를 사용하면 좋다.<br>

랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아닌, 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다.

먼저 싸이파이에서 2개의 확률 분포 클래스를 임포트해보자.<br>

note) 싸이파이(scipy)란><br>
파이썬의 핵심 과학 라이브러리 중 하나로, 적분, 보간, 선형 대수, 확률 등을 포함한 수치 계산 전용 라이브러리

In [27]:
from scipy.stats import uniform, randint

싸이파이의 stats 서브 패키지에 있는 uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다. 이를 '균등 분포에서 샘플링한다'라고 말한다.<br>
randint는 정숫값을 뽑고, uniform은 실숫값을 뽑는다. (사용법은 동일)

In [28]:
# 0에서 10 사이의 범위를 갖는 randint 객체를 만들고 10개의 숫자를 샘플링
rgen = randint(0, 10) # scipy.stats.randint() : 샘플링할 숫자의 범위 지정
rgen.rvs(10) # scipy.stats.randint.rvs() : 샘플링할 숫자의 개수 지정

array([7, 7, 7, 3, 4, 2, 4, 5, 6, 6])

In [29]:
# 0에서 10 사이의 10개의 실수 샘플링
ugen = uniform(0, 10)
ugen.rvs(10)

array([3.47191468, 9.14793867, 3.43816058, 6.89706534, 7.21657865,
       9.55133766, 6.47050094, 4.43768729, 3.08236626, 8.0856235 ])

이런 식으로 랜덤 서치에 randint와 uniform 클래스 객체를 넘겨주고 총 몇 번을 샘플링해서 최적의 매개변수를 찾으라고 명령할 수 있다.

기존 매개변수에 min_samples_leaf 매개변수를 탐색 대상에 추가하여 매개변수 딕셔너리 생성해보기<br>
min_samples_leaf : 리프 노드가 되기 위한 최소 샘플의 개수 지정(어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할 x)

In [30]:
# 매개변수 딕셔너리 생성
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 25),
          'min_samples_leaf' : randint(1, 25)}

샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에서 지정

In [31]:
# 랜덤 서치 수행
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7924af943d90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7924af943d60>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7924afdc67d0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7924af9414e0>},
                   random_state=42)

In [32]:
# 결과 확인
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [33]:
# 최고 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [34]:
# best_estimator_ 속성에 저장된 최적의 모델을 활용하여 테스트 세트 성능 확인
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


In [36]:
# 확인 문제 3번
# 마지막 RandomizedSearchCV 예제에서 DecisionTreeClassifier 클래스에 splitter='random' 매개변수를 추가하고 다시 훈련해보자
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
print('최적 파라미터 :', gs.best_params_)
print('최고 교차 검증 점수 :', np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
print('테스트 세트 성능 :', dt.score(test_input, test_target))

최적 파라미터 : {'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
최고 교차 검증 점수 : 0.8458726956392981
테스트 세트 성능 : 0.786923076923077


테스트 성능 점수가 오히려 낮게 나왔다. 이유는?<br>
결정 트리의 노드를 랜덤하게 분할하기 때문에 100번의 반복에서 최적의 매개변수 조합을 찾지 못 했기 때문